In [1]:
import xarray as xr
%load_ext autoreload
%autoreload 2
import xarray as xr
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import statsmodels.api as sm
#set directory for importinh my_tools,py
import sys
sys.path.append('/home/onno/Thesis/Scripts')
import my_tools
from my_tools import plot_dic, file_dic
from mpl_toolkits.basemap import Basemap, addcyclic
from datetime import datetime
import glob
import matplotlib
matplotlib.use('agg')
%matplotlib qt

In [3]:
path = '/media/onno/Algemeen/Thesis/GFS_performace/'


Average error for each quartile using data between 25 and 75 degrees north or south

In [4]:
init_time = pd.Timestamp('1984-12-01')
variablez = ['envelope','phasespeed','u_wind']
latz_hem = [(75,25),(-25,-75)]
for variable in variablez:
    for lat_hem in latz_hem:
        for i in range(11):
            if (i==0)&(variable=='phasespeed'):
                continue
            fig,ax = plt.subplots()
            file = 'GFS_resampled_RMSE_{}_day_{:02d}_alt.nc'.format(variable,i)
            ds = xr.open_dataset(path+file,decode_times=False)
            ds['time'] =[init_time+pd.Timedelta(i,'days') for i in ds.time.values]
            ds = ds.sel(lat=slice(lat_hem[0],lat_hem[1]))
            #ds_mean = ds.mean(dim=['lat','lon'])
            ds_mean = my_tools.weighted_average_area_3D(ds,variable)
            ds_mean = ds_mean.assign_coords(index=('time',np.arange(140)//4))
            ds_mean_year = ds_mean.groupby(ds_mean.index).mean()
            ax.scatter(np.arange(1985,2020),ds_mean_year.values)
            ax.set_ylabel('RMSE {}'.format(plot_dic[variable]['label']))
            ax.set_xlabel('Year')
            if lat_hem == (75,25):
                fig.suptitle('{} RMSE per quartile Northern Hemisphere lead day {}'.format(plot_dic[variable]['title'],i))
                fig.savefig(path+file[:-3]+'_NH.png',dpi=100)
            else:
                fig.suptitle('{} RMSE per quartile Southern Hemisphere lead day {}'.format(plot_dic[variable]['title'],i))
                fig.savefig(path+file[:-3]+'_SH.png',dpi=100)
            plt.close(fig)

Calculate RMSE for Atlantic Ocean

In [6]:
init_time = pd.Timestamp('1984-12-01')
variablez = ['envelope','phasespeed','u_wind']
latz_hem = [(75,25)]
for variable in variablez:
    for lat_hem in latz_hem:
        for i in range(11):
            if (i==0)&(variable=='phasespeed'):
                continue
            fig,ax = plt.subplots()
            file = 'GFS_resampled_RMSE_{}_day_{:02d}_alt.nc'.format(variable,i)
            ds = xr.open_dataset(path+file,decode_times=False).sel(lat=slice(75,25),lon=slice(300,358))
            ds['time'] =[init_time+pd.Timedelta(i,'days') for i in ds.time.values]
            ds = ds.sel(lat=slice(lat_hem[0],lat_hem[1]))
            #ds_mean = ds.mean(dim=['lat','lon'])
            ds_mean = my_tools.weighted_average_area_3D(ds,variable)
            ds_mean = ds_mean.assign_coords(index=('time',np.arange(140)//4))
            ds_mean_year = ds_mean.groupby(ds_mean.index).mean()
            ax.scatter(np.arange(1985,2020),ds_mean_year.values)
            ax.set_ylabel('RMSE {}'.format(plot_dic[variable]['label']))
            ax.set_xlabel('Year')
            fig.suptitle('{} RMSE per quartile Northern Hemisphere lead day {}'.format(plot_dic[variable]['title'],i))
            fig.savefig(path+file[:-3]+'_Atlantic.png',dpi=100)
            plt.close(fig)

Calculate Statistical Parameters

In [38]:
#do linear regression on all resampled datasets
#create linear regression model
init_time = pd.Timestamp('1984-12-01')
variablez = ['envelope','phasespeed','u_wind']
latz_hem = [(75,25),(-25,-75)]
for variable in variablez:
    for lat_hem in latz_hem:
        df = pd.DataFrame(index=range(11),columns=['Mean Error','Slope','p-value'])
        for i in range(11):
            if (i==0)&(variable=='phasespeed'):
                continue
            file = 'GFS_resampled_RMSE_{}_day_{:02d}.nc'.format(variable,i)
            ds = xr.open_dataset(path+file, decode_times=False)
            ds['time'] =[init_time+pd.Timedelta(i,'days') for i in ds.time.values]
            ds = ds.sel(lat=slice(lat_hem[0],lat_hem[1]))
            ds_mean = my_tools.weighted_average_area_3D(ds,variable)
            df_mean = ds_mean.to_dataframe(name='error')
            df_mean.index = np.arange(len(df_mean.index))
            X = df_mean.index
            X = sm.add_constant(X)
            Y = df_mean['error']
            model = sm.OLS(Y,X).fit()
            p_value = model.pvalues[1]
            slope = model.params[1]
            mean = df_mean['error'].mean()
            df.loc[i,:]=[mean,slope,p_value]
        if lat_hem == (75,25):
            df.to_csv(path+'trend_stats_{}_NH_RMSE.txt'.format(variable))
        else:
            df.to_csv(path+'trend_stats_{}_SH_RMSE.txt'.format(variable))
        

In [32]:
path = '/media/onno/Algemeen/Thesis/GFS_performace/'
file = 'GFS_resampled_RMSE_envelope_day_{:02d}_alt.nc'
init_time = pd.Timestamp('1984-12-01')
days = [0,1,2,3,4,5]
fig,axz = plt.subplots(2,3,figsize=(16,9))
fig.subplots_adjust(hspace=0.2,wspace=0.2,left=0.1,right=0.9,top=0.90,bottom=0.08)
# ax.set_ylim([0,20])
for i,day in enumerate(days):
    ax = axz.flat[i]
    ds = xr.open_dataset(path+file.format(day),decode_times=False).sel(lat=slice(75,25),lon=slice(300,358))
    ds['time'] =[init_time+pd.Timedelta(i,'days') for i in ds.time.values]
    ds = ds.sel(lat=slice(75,25))
    ds_mean = my_tools.weighted_average_area_3D(ds,'envelope')
    ds_mean = ds_mean.assign_coords(index=('time',np.arange(140)//4))
    ds_mean_year = ds_mean.groupby(ds_mean.index).mean()
    ax.scatter(np.arange(1985,2020),ds_mean_year.values)
    ax.set_title('Forecast Day {}'.format(i))
    if i%3==0:
        ax.set_ylabel('Envelope RMSE(m/s)')
    if i>=3:
        ax.set_xlabel('Year')
        
fig.suptitle('GEFS forecast Root Mean Square Error regarding RWP Envelope compared to ERA5 (25-75 °N, 0-60 °W)')
fig.savefig('/media/onno/Algemeen/Thesis/Main_Figures/RMSE.png',dpi=150)